In [9]:
import os     ## Allows operating system commands to be performed though Python
import sys    ## Allows you to exit a Python script for testing
import  glob  ## Global searching of strings as files
import shutil ## Allows movement of files from one directory to another
import string ## Fancy string manipulations
import  glob  ## Global searching of strings as files
import pandas as pd

In [12]:
#change directory to folder with downloaded pdf's
os.chdir('C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Sun\\North_American_MSDS')

In [13]:
pwd=os.getcwd()

In [14]:
#set exec path to folder with pdf-to-text application
execpath = 'C:\\Users\\akomandu\\PDF_Scrapes\\xpdfbin-win-3.04\\bin64\\'

In [15]:
#set names of applications 
htmlexec = "pdftohtml.exe"
txtexec = 'pdftotext.exe'

In [50]:
#set files to convert
files = glob.glob('C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Sun\\North_American_MSDS\\*.pdf')

In [51]:
#convert pdf files to text files
for f in files:
    filename = os.path.join("C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Sun\\North_American_MSDS", os.path.basename(f))
    dirname = os.path.basename(f)[:-4]
    
    txtcmd = os.path.join(execpath,txtexec)

    txtcmd = " ".join([txtcmd,"-table","-nopgbrk",f])

#     print(os.path.isfile(filename))
#    print(" ".join([txtcmd, filename]))
    
#     print(txtcmd)
    #print(os.path.isfile(os.path.join("C:\\Users\\akomandu\\PDF Scrapes\\Georgia Pacific MSDS",os.path.basename(filename))))
#     print(dirname)

    os.system(txtcmd)
#    os.system(" ".join([txtcmd, filename]))
    #textpath = os.path.join(pwd,"PDF Scrapes","Georgia Pacific MSDS", dirname  + " .txt ")
    
    #if (os.path.isfile(textpath)):
        #shutil.move(textpath,os.path.join(pwd,"text",dirname +".txt"))
    #else:
        ## Throw an error if the path doesn't exist
        #sys.exit("Path Not Found")


In [52]:
#create function for cleaning lines
clean = lambda dirty: ''.join(filter(string.printable.__contains__, dirty))

In [53]:
#find all text files
#set path to text files
pwd = ['C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Sun\\North_American_MSDS']
os.chdir("".join(pwd))
pwd = os.getcwd()

In [54]:
#find all text files
text_files = glob.glob('*.txt')

In [60]:
#create lists for pertinent information in each text file
Product_Generic_Name = []
Chemical_Name = []
CAS_Number = []
Percentage = []
Product_Label_Name = []
UPC_Code = []
Product_Code = []
File = []

#iterate through each text file to clean lines and append relevant information to lists
for file in text_files:
    
    ifile= open(file)
    for line in ifile:
        if line == '/n': continue
        cline = clean(line)
        cline = cline.lower()
        cline = cline.replace(',','_')
        cline = cline.replace(';','_')
        cline = cline.strip()

        tline = "".join(cline.split())
        tline = "".join(cline.split())
        cline = cline.split("  ")
        cline = [x.strip() for x in cline if x != ""]

        if 'upc code(s)' in cline:
            upc_coded = (cline[1:])
            upc_code = ''.join(upc_coded)
        elif 'upc code(s)' in cline:
            code = (cline[-1])
        elif 'product code' in cline:
            pcode = cline[-1]
        elif 'product label name' in cline:
            label_named = (cline[1:])
            label_name = ''.join(label_named)
            
        elif 'product generic name' in cline:
            name = (cline[-1])

        elif 'cas-no' in cline:
            for line in ifile:
                dline = clean(line)
                dline = dline.lower()
                dline = dline.replace(',','_')
                dline = dline.replace(';','_')
                dline = dline.strip()
                dline  = dline.split("  ")
                dline = [x.strip() for x in dline if x != ""]

                list1 = []
                list1.append(dline)
                list2 = []
                list2 = [x for x in list1 if x != []]

                if 'trade secret' in str(line).lower():
                    break
                if 'consumer household use' in str(line).lower():
                    break
                if 'first aid measures' in str(line).lower():
                    break

                for i in list2:
                    try:
                        File.append(file)
                        Product_Generic_Name.append(name)
                    except IndexError:
                        File.append(' ')
                        Product_Generic_Name.append(' ')
                    try:
                        UPC_Code.append(upc_code)
                    except IndexError:
                        UPC_Code.append(' ')
                    try:
                        Product_Code.append(pcode)
                    except IndexError:
                        Product_Code.append('')
                    try:
                        Product_Label_Name.append(label_name)
                    except:
                        Product_Label_Name.append(' ')
                    try:
                        Chemical_Name.append(i[0])
                    except IndexError:
                        Chemical_Name.append(' ')
                    try:
                        CAS_Number.append(i[1])
                    except IndexError:
                        CAS_Number.append(' ')
                    try :
                        Percentage.append(i[2])
                    except IndexError:
                        Percentage.append(' ')
                    finally:
                        pcode = ''

                

In [61]:
scraped = pd.DataFrame({"File":File, 'Product_Generic_Name': Product_Generic_Name, 'Product_Label_Name': Product_Label_Name, 'UPC_Code': UPC_Code, 'Product_Code': Product_Code, 'Chemical Name': Chemical_Name, 'CAS_Number': CAS_Number, 'Percentage': Percentage})
great = scraped.drop_duplicates(keep='first')

In [71]:
final = great[4:]

In [72]:
#create function to export dataframe to excel file
def checkfile(file_name):
    """
    Usage:
    ------
    checkfile(file_name)

    Description:
    ------------
    Function to check if a file name exists in the current working
    directory and rename the new file, rather than overwrite the old file
    with new data.

    Parameters:
    -----------
    file_name : string, database name tag in .my.cnf file (default=None)
    out       : string, a unique file name based on the provided file_name;
                uniqueness is insured by incrementing a number appended to the
                file by 1
    """
    cwd = os.getcwd()
    cwf = os.path.join(cwd,file_name)
    if not os.path.exists(cwf):
        return cwf
    root, ext = os.path.splitext(os.path.expanduser(cwf))
    dir = os.path.dirname(root)
    fname = os.path.basename(root)
    candidate = fname+ext
    index = 1
    ls = set(os.listdir(dir))
    while candidate in ls:
        candidate = "{}_{}{}".format(fname,index,ext)
        index += 1
    return os.path.join(dir,candidate)

def to_excel(df,xlfile,sheet_name='Sheet 1',cas_col='casrn',index=False,filecheck=False,encoding=None):
    """
    Usage:
    ------
    to_excel(df,xlfile)

    Description:
    ------------
    Writes Pandas dataframe to MS Excel file. This differs from Pandas' own
    to_excel function in that it will find the 'casrn' column and write it to
    the file as a string instead of a number, thus preventing some CASRNs to be
    autoformatted to strings.

    Parameters:
    -----------
    df              pandas.DataFrame, object to write to an MS Excel file
    xlfile          string, name of MS Excel file to generate
    sheet_name      string, name of MS Excel sheet the dataframe will be written to
    cas_col         string, name of the column in the dataframe containing CASRNs (default:'casrn')
    index           boolean, default True write row names (index)
    checkfile       check if a file by the same name already exists in the path

    """
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    if cas_col not in df.columns.tolist():
        print("Error! 'casrn' column not in dataframe. Use Pandas' to_excel function.")
        return

    cas_col = alphabet[df.columns.get_loc(cas_col)].upper()
    if filecheck:
        writer = pd.ExcelWriter(checkfile(xlfile),engine="xlsxwriter")
    else:
        writer = pd.ExcelWriter(xlfile,engine="xlsxwriter")
    df.to_excel(writer,sheet_name=sheet_name,index=index,encoding=encoding)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    format1 = workbook.add_format({"bold":False,"font_color":"black"})
    worksheet.set_column(cas_col+':'+cas_col,12,format1)
    writer.save()
    return


In [73]:
#had to manually delete entries in excel file
to_excel(df = final, xlfile= 'Sun_NA_MSDS_07232018.xlsx', cas_col = 'CAS_Number')